In [8]:
DATA_PATH = r"C:\Users\Mohamed Gad\OneDrive\Desktop\All About Data Science\Data Sciience Projects\End to End - Ai-Pro\V1\iti project\AI-Pro Project\data\raw\survey_results_public.csv "
EXPORT_PATH = r"C:\Users\Mohamed Gad\OneDrive\Desktop\All About Data Science\Data Sciience Projects\End to End - Ai-Pro\V1\iti project\AI-Pro Project\data\processed\1_preprcoessed_df.pkl"


In [9]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
raw_data = pd.read_csv(DATA_PATH)

## Functtions 

In [10]:

def split_answers(data_series, delimiter=';'):
    """Split multiple answers in a single string to a list of single strings each representing a single answer
    
    Args:
        data_series (pandas.Series): Series containing the data to split
        delimiter (str): Delimiter used to separate answers. Defaults to ';'.
    
    Returns:
        pandas.Series: Series with split answers
    
    """
    
    # Sub functions
    def is_splittable(pd_series, delimiter):
        """Function to check if a string can be split"""
        return pd_series.str.contains(delimiter, na=False)
    
    def split_answer(pd_series, delimiter):
        """Function to split a single answer"""
        return pd_series.str.split(delimiter)
    
    # Fill NaN values with empty string
    data_series = data_series.fillna('')
    
    # Get the indices of the NaN values
    nan_indices = data_series.index[data_series.isna()]
    
    # Check if the series needs to be split
    if is_splittable(data_series, delimiter).any():
        # Split the non-NaN values using the delimiter
        split_series = split_answer(data_series[data_series.notna()],delimiter)
        
        
        # Insert the NaN values back into the split series
        split_series = split_series.reindex(data_series.index)
        split_series.loc[nan_indices] = np.nan
        
        return split_series
    else:
        # The series does not need to be split
        return data_series
    
def parse_age (x) :
    str_dct ={'More than 50 years' : 50,
              'Less than 1 year' : 1,
              'Younger than 5 years' : 1,
              'Older than 85' : 85}
    
    if isinstance(x, str) and x in str_dct :
        return str_dct[x]
    else :
        return x
    


### Check Results 

In [11]:
# Check the split answers results in a sepecific column
print  (raw_data['LanguageWorkedWith'].iloc[0])
print (split_answers(raw_data['LanguageWorkedWith']).iloc[0])

C#;HTML/CSS;JavaScript
['C#', 'HTML/CSS', 'JavaScript']


In [12]:
#apply to all object columns 
# Create a boolean mask for object columns
mask = raw_data.dtypes == object

# Apply the function to only object columns
raw_data.loc[:, mask] = raw_data.loc[:, mask].apply(split_answers)


In [27]:
raw_data.sample(1).iloc[0]

Respondent                                                      55335
MainBranch                             I am a developer by profession
Hobbyist                                                           No
Age                                                               NaN
Age1stCode                                                         20
                                           ...                       
WebframeWorkedWith    [Angular.js, jQuery, Laravel, React.js, Vue.js]
WelcomeChange                                                        
WorkWeekHrs                                                      45.0
YearsCode                                                           8
YearsCodePro                                                        4
Name: 54230, Length: 61, dtype: object

In [15]:
# Check the age parsing
print (raw_data['YearsCodePro'].iloc[82])
print (raw_data['YearsCodePro'].apply(parse_age).iloc[82])

Less than 1 year
1


##### Applying the changes to the data and save it as a pickled file

In [16]:
# now after making sure that every thing work well let's apply them 
df = raw_data.copy()
for col in raw_data.columns :
    # making sure that the type of the column in str (object)
    if df.dtypes[col] != df.dtypes['LanguageWorkedWith'] :
        continue
    df[col] = split_answers (raw_data[col])


In [17]:
df[['YearsCodePro', 'YearsCode', 'Age1stCode']] = df[['YearsCodePro', 'YearsCode', 'Age1stCode']].applymap(parse_age) # applymap to apply a function to each single variable not to a pandas series
df[['YearsCodePro', 'YearsCode', 'Age1stCode']] = df[['YearsCodePro', 'YearsCode', 'Age1stCode']].apply(pd.to_numeric)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Respondent                    64461 non-null  int64  
 1   MainBranch                    64461 non-null  object 
 2   Hobbyist                      64461 non-null  object 
 3   Age                           45446 non-null  float64
 4   Age1stCode                    57900 non-null  float64
 5   CompFreq                      64461 non-null  object 
 6   CompTotal                     34826 non-null  float64
 7   ConvertedComp                 34756 non-null  float64
 8   Country                       64461 non-null  object 
 9   CurrencyDesc                  64461 non-null  object 
 10  CurrencySymbol                64461 non-null  object 
 11  DatabaseDesireNextYear        64461 non-null  object 
 12  DatabaseWorkedWith            64461 non-null  object 
 13  D

 #### Now Every thing seems good let's save the data as a piickled file 

In [19]:
df.to_pickle(EXPORT_PATH)